In [2]:
import time
import pandas as pd
import numpy as np
import sys
from time import perf_counter
sys.path.append('/Users/ziniuwu/Desktop/research/BayesNet')
from Evaluation.utils import parse_query, save_csv

In [3]:
true_cardinalities_path = '/Users/ziniuwu/Desktop/research/deepdb-public/benchmarks/job-light/sql/job_light_true_cardinalities.csv'
query_filename = '/Users/ziniuwu/Desktop/research/deepdb-public/benchmarks/job-light/sql/job_light_queries.sql'
df_true_card = pd.read_csv(true_cardinalities_path)
with open(query_filename) as f:
    queries = f.readlines()

In [4]:
from Schemas.imdb.schema import gen_job_light_imdb_schema
schema = gen_job_light_imdb_schema(' ')

query_no = 0
query_str = queries[0]
query_str = query_str.strip()
print(f"Predicting cardinality for query {query_no}: {query_str}")

query = parse_query(query_str.strip(), schema)

cardinality_true = df_true_card.loc[df_true_card['query_no'] == query_no, ['cardinality_true']].values[0][0]
print()

Predicting cardinality for query 0: SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=112 AND mc.company_type_id=2;



In [7]:
query = """SELECT COUNT(*) FROM movie_companies mc,title t,movie_info_idx mi_idx WHERE t.id=mc.movie_id AND t.id=mi_idx.movie_id AND mi_idx.info_type_id=112 AND mc.company_type_id in (2,3,4)"""

In [8]:
query_s = parse_query(query.strip(), schema)

In [9]:
print(query_s.relationship_set)
print(query_s.conditions)
print(query_s.table_where_condition_dict)

{'movie_info_idx.movie_id = title.id', 'movie_companies.movie_id = title.id'}
[('movie_companies', 'company_type_id IN (2,3,4)'), ('movie_info_idx', 'info_type_id=112')]
{'movie_companies': ['company_type_id IN (2,3,4)'], 'movie_info_idx': ['info_type_id=112']}


In [11]:
for relationship_obj in schema.relationships:
    rel = relationship_obj.identifier
    print(rel)

movie_info_idx.movie_id = title.id
movie_info.movie_id = title.id
cast_info.movie_id = title.id
movie_keyword.movie_id = title.id
movie_companies.movie_id = title.id


In [16]:
import ast
ast.literal_eval("(a,b,c)")

ValueError: malformed node or string: <_ast.Name object at 0x119ae98d0>

In [15]:
type(ast.literal_eval("abs"))

ValueError: malformed node or string: <_ast.Name object at 0x119b14090>

In [17]:
s="(a,c)"

In [20]:
s[1:][:-1].split(',')

['a', 'c']